# Interpolacja 

### Zadanie 1
Stablicuj następujące funkcje: sqrt(x), sin(x), x^3+2x w czterech punktach należących do przedziału 0 do 10.

In [65]:
import numpy as np
from tabulate import tabulate


def lookupTab(fx, steps, x_start = 0, x_end = 10):
    array_x = np.linspace(x_start, x_end, steps)
    array_y = np.zeros([steps])
    for i in range(len(array_x)):
        array_y[i]=fx(array_x[i])
    return list(zip(array_x, array_y))


print(tabulate({"x": list(zip(*lookupTab(lambda x : np.sqrt(x), 4 )))[0], "sqrt(x)": list(zip(*lookupTab(lambda x : np.sqrt(x), 4)))[1]}, headers="keys", floatfmt=".10f"),"\n")
print(tabulate({"x": list(zip(*lookupTab(lambda x : np.sin(x), 4 )))[0], "sin(x)": list(zip(*lookupTab(lambda x : np.sin(x), 4)))[1]}, headers="keys", floatfmt=".10f"),"\n")
print(tabulate({"x": list(zip(*lookupTab(lambda x : x**3+2*x, 4 )))[0], "x^3+2x ": list(zip(*lookupTab(lambda x : x**3+2*x, 4)))[1]}, headers="keys", floatfmt=".10f"),"\n")

            x       sqrt(x)
-------------  ------------
 0.0000000000  0.0000000000
 3.3333333333  1.8257418584
 6.6666666667  2.5819888975
10.0000000000  3.1622776602 

            x         sin(x)
-------------  -------------
 0.0000000000   0.0000000000
 3.3333333333  -0.1905679629
 6.6666666667   0.3741512306
10.0000000000  -0.5440211109 

            x          x^3+2x
-------------  ---------------
 0.0000000000     0.0000000000
 3.3333333333    43.7037037037
 6.6666666667   309.6296296296
10.0000000000  1020.0000000000 



### Zadanie 2 
Napisz algorytm znajdujący wielomian interpolujący Lagrange dla powyższych stablicowanych funkcji.

In [66]:
import numpy as np
import scipy
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial

def interpolate(inpts):
    n = len(inpts)
    thepoly = n*[0]
    for i in range(len(inpts)):
        prod = 1
        for j in (j for j in range(n) if (j != i)):
            prod *= (inpts[i][0] - inpts[j][0])
        prod = inpts[i][1]/prod
        theterm = [prod] + (n-1)*[0]
        for j in (j for j in range(n) if (j != i)):
            for k in range(n-1,0,-1):
                theterm[k] += theterm[k-1]
                theterm[k-1] *= (-inpts[j][0])
        for j in range(n):
            thepoly[j] += theterm[j]
    return(thepoly)

x = np.array(np.linspace(0,10,4))

print("Sqrt(x):")
Array1=lookupTab(lambda x: np.sqrt(x),4)
print("My method:", interpolate(Array1))

#Comparison with predefined function from scipy
y = np.sqrt(x)
poly = lagrange(x, y)
P=Polynomial(poly).coef
P=P[::-1]
print("Predefined method from scipy:", P)

print("Sin(x):")
Array2=lookupTab(lambda x: np.sin(x),4)
print("My method:", interpolate(Array2))

#Comparison with predefined function from scipy
y = np.sin(x)
poly = lagrange(x, y)
P=Polynomial(poly).coef
P=P[::-1]
print("Predefined method from scipy:", P)

print("x^3+2x:")
Array3=lookupTab(lambda x: x**3+2*x,4)
print("My method:", interpolate(Array3))

#Comparison with predefined function from scipy
y = x**3+2*x
poly = lagrange(x, y)
P=Polynomial(poly).coef
P=P[::-1]
print("Predefined method from scipy:",P )




Sqrt(x):
My method: [0.0, 0.7975004346701113, -0.08833641129156172, 0.004020914442623438]
Predefined method from scipy: [ 0.          0.79750043 -0.08833641  0.00402091]
Sin(x):
My method: [0.0, -0.39428133143392285, 0.13470596313982547, -0.010071804110532687]
Predefined method from scipy: [ 0.         -0.39428133  0.13470596 -0.0100718 ]
x^3+2x:
My method: [0.0, 2.000000000000014, 0.0, 1.0]
Predefined method from scipy: [ 0.00000000e+00  2.00000000e+00 -7.10542736e-15  1.00000000e+00]


## **Zadanie 3**

Porównaj wartość dokładną z wynikiem interpolacji dla punktów znajdujących się pomiędzy węzłami wielomianu (w połowie odległości) interpolującego. Oszacuj dokładność interpolacji. 


In [81]:
#a0 + a1x + a2x2 + ... + an-1xn-1
import statistics

def fillArray(steps):
    arr1=np.linspace(0, 10, steps)
    arr2=np.zeros([len(arr1)-1])
    
    for i in range(len(arr2)):
        arr2[i]=(arr1[i+1]-arr1[i])/2+arr1[i]
    return arr2

def interpolationResults(fx, steps):
    demi_points=fillArray(steps)
    lookupTabGivenFx=lookupTab(fx, steps)
    #y_values=list(zip(*lookupTab(fx, steps)))[1]
    results=np.zeros([len(demi_points)])
    for i in range(len(demi_points)):
        coef=interpolate(lookupTabGivenFx)
        for j in range(len(coef)):
            results[i]+=coef[j]*(demi_points[i]**j)
    return results
        
        
def normalResults(fx, steps):
    demi_points=fillArray(steps)
    y=fx
    results=np.zeros([len(demi_points)])
    for i in range(len(demi_points)):
        results[i]=y(demi_points[i])
        
    return results

def calculateError(fx, steps):
    normal_results = normalResults(fx, steps)
    interpolation_results = interpolationResults(fx, steps)
    absolute_errors=np.zeros([len(normal_results)])
    relative_errors=np.zeros([len(normal_results)])
     
    for i in range(len(absolute_errors)):
        absolute_errors[i]=abs(interpolation_results[i]-normal_results[i])
        relative_errors[i]=abs((interpolation_results[i]-normal_results[i])/normal_results[i])
    print(tabulate({"actual values ": normal_results," interpolation": interpolation_results, 
                  "realtive error": relative_errors, "absolute error": absolute_errors}, headers="keys", floatfmt=".10f"))

    
    
print("\nResults for function sqrt(x)\n")
calculateError(lambda x: np.sqrt(x),4)

print("\nResults for function sin(x)\n")
calculateError(lambda x: np.sin(x),4)

print("\nResults for function x^3+2x)\n")
calculateError(lambda x: x**3+2*x, 4)



Results for function sqrt(x)

  actual values      interpolation    realtive error    absolute error
----------------  ----------------  ----------------  ----------------
    1.2909944487      0.0000000000      1.0000000000      1.2909944487
    2.2360679775      4.1658065112      0.8630053080      1.9297385337
    2.8867513459      7.3284126879      1.5386366229      4.4416613420

Results for function sin(x)

  actual values      interpolation    realtive error    absolute error
----------------  ----------------  ----------------  ----------------
    0.9954079578      0.0000000000      1.0000000000      0.9954079578
   -0.9589242747     -0.0261586749      0.9727208127      0.9327655997
    0.8872941081      0.0899132812      0.8986657520      0.7973808269

Results for function x^3+2x)

  actual values       interpolation    realtive error     absolute error
----------------  -----------------  ----------------  -----------------
    7.9629629630       0.0000000000      1.000000000

### Zadanie 4
Powtórz powyższe kroki dla 3, 5 i 8 węzłów interpolacji - podsumuj badania. 

In [79]:
print("3 interpolation nodes:\n")
print("\nResults for function sqrt(x)\n")
calculateError(lambda x: np.sqrt(x),3)

print("\nResults for function sin(x)\n")
calculateError(lambda x: np.sin(x),3)

print("\nResults for function x^3+2x)\n")
calculateError(lambda x: x**3+2*x, 3)

print("\n\n5 interpolation nodes:\n")
print("\nResults for function sqrt(x)\n")
calculateError(lambda x: np.sqrt(x),5)

print("\nResults for function sin(x)\n")
calculateError(lambda x: np.sin(x),5)

print("\nResults for function x^3+2x)\n")
calculateError(lambda x: x**3+2*x, 5)

print("\n\n8 interpolation nodes:\n")
print("\nResults for function sqrt(x)\n")
calculateError(lambda x: np.sqrt(x),8)

print("\nResults for function sin(x)\n")
calculateError(lambda x: np.sin(x),8)

print("\nResults for function x^3+2x)\n")
calculateError(lambda x: x**3+2*x, 8)

3 interpolation nodes:


Results for function sqrt(x)

  actual values      interpolation    realtive error    absolute error
----------------  ----------------  ----------------  ----------------
    1.5811388301      1.2817662756      0.1893398282      0.2993725545
    2.7386127875      2.8629051057      0.0453851376      0.1242923182

Results for function sin(x)

  actual values      interpolation    realtive error    absolute error
----------------  ----------------  ----------------  ----------------
    0.5984721441     -0.6511905671      2.0880883489      1.2496627112
    0.9379999768     -0.9232011226      1.9842229696      1.8612010994

Results for function x^3+2x)

  actual values      interpolation    realtive error    absolute error
----------------  ----------------  ----------------  ----------------
   20.6250000000    -26.2500000000      2.2727272727     46.8750000000
  436.8750000000    483.7500000000      0.1072961373     46.8750000000


5 interpolation nodes:


Resul

 # Wnioski
 
Im więcej węzłów interpolacji tym dokładniej wielomian Lagrange'a odzwierciedla funkcje. 
Najdokładniejsze wyniki otrzymaliśmy w przypadku funkcji x^3 + 2x. Wynika to z tego że jest to wielomian przez co oszacowanie go przez wielomian Lagrange'a jest bardzo dokładne